# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

Датафрейм переработан и готов к использованию. Есть даже значения целевого признака is_ultra. Все остальные признаки подходят для обучения моделей по определению целевого признака (тарифа) для каждого объекта (пользователя). 

## 2. Разбейте данные на выборки

In [5]:
df_train, df_others = train_test_split(df, test_size=0.40, random_state=12345)

In [6]:
df_valid, df_test = train_test_split(df_others, test_size=0.50, random_state=12345)

In [7]:
df_train.shape

(1928, 5)

In [8]:
df_valid.shape

(643, 5)

In [9]:
df_test.shape

(643, 5)

### Вопрос 
Как я понял, должно  быть три выборки. Отдельного датафрейма с данными для тестирования в задании я не обнаружил. Если так, то какой смысл кроме обучающей и валидационной выделять тестировочную выборку из тех же исходных данных, если уже есть валидационная ?

## 3. Исследуйте модели

Выберем оптимальное решающее дерево и на его основе его параметров постоим случайный лес. Далее проверим модель логичтической регрессии

In [10]:
train_features = df_train.drop(['is_ultra'], axis = 1)
train_target = df_train['is_ultra']

In [11]:
valid_features = df_valid.drop(['is_ultra'], axis = 1)
valid_target = df_valid['is_ultra']

In [12]:
t_models = [] 
for i in range(1, 30, 1):
    model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = i) #
    model_tree.fit(train_features, train_target)
    predictions =   model_tree.predict(valid_features) # проверим модель на валидационной выборке
    accuracy = accuracy_score(valid_target, predictions)
    t_models.append({
        'depth': i,
        'accuracy': accuracy,
        'model': model_tree
    })
    print("max_depth: ", i, '; ', 'accuracy = ', accuracy)

max_depth:  1 ;  accuracy =  0.7542768273716952
max_depth:  2 ;  accuracy =  0.7822706065318819
max_depth:  3 ;  accuracy =  0.7853810264385692
max_depth:  4 ;  accuracy =  0.7791601866251944
max_depth:  5 ;  accuracy =  0.7791601866251944
max_depth:  6 ;  accuracy =  0.7838258164852255
max_depth:  7 ;  accuracy =  0.7822706065318819
max_depth:  8 ;  accuracy =  0.7791601866251944
max_depth:  9 ;  accuracy =  0.7822706065318819
max_depth:  10 ;  accuracy =  0.7744945567651633
max_depth:  11 ;  accuracy =  0.7620528771384136
max_depth:  12 ;  accuracy =  0.7620528771384136
max_depth:  13 ;  accuracy =  0.7558320373250389
max_depth:  14 ;  accuracy =  0.7589424572317263
max_depth:  15 ;  accuracy =  0.7465007776049767
max_depth:  16 ;  accuracy =  0.7340590979782271
max_depth:  17 ;  accuracy =  0.7356143079315708
max_depth:  18 ;  accuracy =  0.7309486780715396
max_depth:  19 ;  accuracy =  0.7278382581648523
max_depth:  20 ;  accuracy =  0.7216174183514774
max_depth:  21 ;  accuracy = 

Не брал глубину дерева больше 30, чтобы не переобучить модель. 

In [19]:
#выведем максимальный показатель качества 
best_tree = max(t_models, key = lambda x: x['accuracy'])

In [20]:
best_tree

{'depth': 3,
 'accuracy': 0.7853810264385692,
 'model': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=12345, splitter='best')}

In [21]:
best_tree_model = best_tree['model']

In [25]:
# замерим точность на тестовой выборке
train_predictions = best_tree_model.predict(train_features)
train_tree_accuracy = accuracy_score(train_target, train_predictions)

In [26]:
train_tree_accuracy

0.8075726141078838

Создадим модель логистической регрессии 

In [27]:
model_logistic_regression = LogisticRegression(random_state=12345)
model_logistic_regression.fit(train_features, train_target)
prediction =  model_logistic_regression.predict(valid_features)
accuracy_regression_valid = accuracy_score(valid_target, prediction)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
accuracy_regression_valid

0.7589424572317263

In [46]:
# замерим точность на тестовой выборке
train_predictions = model_logistic_regression.predict(train_features)
train_regression_accuracy = accuracy_score(train_target, train_predictions)

In [47]:
train_regression_accuracy

0.7505186721991701

Логистическая регрессия показала меньшую точность, чем случайный лес на следующих ячейках.

Подберем модель с помощью случайного лучший леса, попробовав разрыне комбинации глубины деревьев, и их количества, чтобы получить усредненный алгаритм для определения тарифа.

In [29]:
f_models = []
for i in range(2, 70, 2):
    for j in range(2,40,2):
        model_random_forest = RandomForestClassifier(n_estimators = i, random_state=12345, max_depth = j)
        model_random_forest.fit(train_features, train_target)
        prediction =  model_random_forest.predict(valid_features)
        accuracy = accuracy_score(valid_target, prediction)
        f_models.append({
            'estimators' : i,
            'max_depth': j,
            'accuracy': accuracy,
            'model': model_random_forest
        })
        print("estimators: ", i, "; max depth: ",j ,'; ', 'accuracy = ', accuracy)

estimators:  2 ; max depth:  2 ;  accuracy =  0.7822706065318819
estimators:  2 ; max depth:  4 ;  accuracy =  0.7776049766718507
estimators:  2 ; max depth:  6 ;  accuracy =  0.7853810264385692
estimators:  2 ; max depth:  8 ;  accuracy =  0.7807153965785381
estimators:  2 ; max depth:  10 ;  accuracy =  0.7698289269051322
estimators:  2 ; max depth:  12 ;  accuracy =  0.7620528771384136
estimators:  2 ; max depth:  14 ;  accuracy =  0.7573872472783826
estimators:  2 ; max depth:  16 ;  accuracy =  0.7356143079315708
estimators:  2 ; max depth:  18 ;  accuracy =  0.7169517884914464
estimators:  2 ; max depth:  20 ;  accuracy =  0.7636080870917574
estimators:  2 ; max depth:  22 ;  accuracy =  0.7573872472783826
estimators:  2 ; max depth:  24 ;  accuracy =  0.7636080870917574
estimators:  2 ; max depth:  26 ;  accuracy =  0.7636080870917574
estimators:  2 ; max depth:  28 ;  accuracy =  0.7636080870917574
estimators:  2 ; max depth:  30 ;  accuracy =  0.7636080870917574
estimators:  2

In [31]:
best_forest = max(f_models, key = lambda x: x['accuracy'])

In [33]:
best_forest

{'estimators': 40,
 'max_depth': 8,
 'accuracy': 0.8087091757387247,
 'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=8, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=40,
                        n_jobs=None, oob_score=False, random_state=12345,
                        verbose=0, warm_start=False)}

In [34]:
# лучший рзультат - лес максимальной глубиной 8. Точнось - 80,87 % 

In [50]:
best_forest_model = best_forest['model'] #лучшая  модель

In [51]:
# замерим точность на тестовой выборке
train_predictions = best_forest_model.predict(train_features)
train_forest_accuracy = accuracy_score(train_target, train_predictions)

In [52]:
train_forest_accuracy

0.875

In [35]:
# Все понятно, спасибо большое, что объяснил 

### Вывод

Я обучил и протестировал на валидационной выборке различне вариации моделей ( найденных с помощью алгоритмов: решающее дерево, случайный лес и логистическая регрессия). Результат метрики точности модели, полученной с помощью  случайного леса оказался самым высоким. Чтоб получить наилучшую в плане точности модель я создал таблицу случайных лесов с различным количеством estimators и разной глубиной деревьев. Сравнив модели лесов в таблице, я получил модель, у которой точность максимальна. Эту модель я назвал 'best_forest_model', точность валидационной выборке - 0.8087.       
    

## 4. Проверьте модель на тестовой выборке

In [37]:
test_features = df_test.drop(['is_ultra'], axis = 1)
test_target = df_test['is_ultra']

Проверим на тестовой выборке лучшую модель

##### ЛЕС

In [38]:
best_forest

{'estimators': 40,
 'max_depth': 8,
 'accuracy': 0.8087091757387247,
 'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=8, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=40,
                        n_jobs=None, oob_score=False, random_state=12345,
                        verbose=0, warm_start=False)}

In [39]:
prediction_forest_test = best_forest_model.predict(test_features)
accuracy_forest_test = accuracy_score(test_target, prediction_forest_test)

In [40]:
accuracy_forest_test

0.7962674961119751

Модель best_forest_model показала наилучший результат точности почти в 0,80

##### ДEРЕВО

In [41]:
best_tree

{'depth': 3,
 'accuracy': 0.7853810264385692,
 'model': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=12345, splitter='best')}

In [42]:
prediction_tree_test = best_tree_model.predict(test_features)
accuracy_tree_test = accuracy_score(test_target, prediction_tree_test)

In [43]:
accuracy_tree_test

0.7791601866251944

Точность модели постороенной с помощию решающего дерева не оказалась выше точности модели, построенной лучшим лесом

##### ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ

In [48]:
prediction_regression = model_logistic_regression.predict(test_features)
accuracy_regression_test = accuracy_score(test_target, prediction_regression)

In [49]:
accuracy_regression_test

0.7402799377916018

Регрессия показала худший результат

### Вывод

На тестовой выборке модель best_forest_model, построенная лучшим лесом оказалось самой точной. Ее точность составила 0,80

### Выводы по переобучению

1. Для дерева:  
    точность на тренировочной выборке - 0.8075726141078838  
    точность на тествой выборке - 0.7791601866251944  
    Вывод -  модель незначительно переобучена   
       
2. Для леса:   
    точность на тренировочной выборке - 0.875  
    точность на тествой выборке - 0.7962674961119751    
    Вывод - модель переобучена   
       
3. Для регрессии:    
    точность на тренировочной выборке - 0.7505186721991701  
    точность на тествой выборке - 0.7402799377916018  
    Вывод - модель обучена оптимально   
    
Хоть модель, созданная c помощью случайного леса и переобучена, на тестовой выборке она дает наилучшую точность. 

## 5. (бонус) Проверьте модели на адекватность

Здесь я сравню результаты точности случайной и константоной моделей, построенных на тех же гиперпараметрах, которые имеет best_forest_model c точностью самой best_forest_model

In [53]:
import random

In [54]:
#добавим в исходный датафрейм столбец и со случайными классами, не зависящими от признаков

In [55]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [56]:
#сравним классы для is_ultra(df_train)  на сбалансированность
pivot_is_ultra = df_train.pivot_table(index = 'is_ultra', values = 'calls', aggfunc = 'count' )

In [57]:
pivot_is_ultra

,calls
is_ultra,
0,1335
1,593


In [58]:
pivot_is_ultra.loc[0, 'calls'] / (pivot_is_ultra.loc[0, 'calls'] + pivot_is_ultra.loc[1, 'calls']) #найдем долю класса 0

0.6924273858921162

соответсвенно 0,69 и 0,31 для is_ultra. Насколько я понял из треда, нужно  случайную модель 
обучить на целевом признаке расставленном ранодомно (is_ultra_random), но с таким же соотношением классов, как и в изначальном целевом признаке (is_ultra)

In [59]:
right_proportion = []

for i in range(0, 1335): #на 1 бошльше конца посл числа
    right_proportion.append(0)
for i in range (0, 593):
    right_proportion.append(1)
    

In [ ]:
random.shuffle(right_proportion) #рандомно перемешаем будущий целефой признак

In [ ]:
len(right_proportion)

In [ ]:
df_train['is_ultra_random'] = right_proportion

In [ ]:
df_train

In [ ]:
pivot_random_is_ultra = df_train.pivot_table(index = 'is_ultra_random', values = 'calls', aggfunc = 'count' )

In [ ]:
pivot_random_is_ultra # как виддно, теперь пропорции такие же 

In [ ]:
#обучим случайную модель
random_train_features = df_train.drop(['is_ultra_random', 'is_ultra'], axis = 1)
random_train_target = df_train['is_ultra_random']
random_model = RandomForestClassifier(n_estimators = 40, random_state=12345, max_depth = 8)
random_model.fit(random_train_features, random_train_target)
# проверим точность случайной модели на тестовой выборке
random_prediction =  random_model.predict(test_features)
random_accuracy = accuracy_score(test_target, random_prediction)

In [ ]:
random_accuracy

In [ ]:
train_target.mean()

In [ ]:
constant_predictions = pd.Series(1, index = test_target.index) #взял индекс 
# от test_target, чтобы не возникло ошибки в constant_accuracy

In [ ]:
#посчитаем точность константной модели
constant_accuracy = accuracy_score(test_target, constant_predictions)

In [ ]:
constant_accuracy

### Вывод 
Результат точности на тестовой выборке исследуемой модели (best_forest_model) оказались выше результата точности случайной модели, который зависел от баланса классов 31% к 69%, следовательно, раз точность исследуемой модели првосходдит точность случайной примерно на 10%, то исследуемая модель адекватна. Константная выборка (если провести инверсию) оказалась сравнима со случайной, что говорит о том, что классы сбалансированы

P.S. Если что-то не так сказал, поправь меня пожалуйста :)

### Общий вывод

best_forest_model - наилучшая модель. Её точность составила почти 80%